In [1]:
# Define the neural network architecture using PyTorch as usual

import torch
import torch.nn as nn

class ReLUConvBN(nn.Module):
    def __init__(self, out_channels, kernel_size, stride, padding):
        super().__init__()

        self.kernel_size = kernel_size
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.LazyConv2d(
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                dilation=2,
                bias=False,
            ),
            nn.LazyBatchNorm2d(affine=True, track_running_stats=True),
        )

    def forward(self, x):
        return self.op(x)


class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x
    


In [54]:
# Define the NEPS space for the neural network architecture

from neps.space.neps_spaces.parameters import Pipeline, Operation, Categorical, Resampled

class NN_Space(Pipeline):
    _id = Operation(operator=Identity)
    _three = Operation(operator=nn.Conv2d,kwargs={"in_channels":3, "out_channels":3, "kernel_size":3, "stride":1, "padding":1})
    _one = Operation(operator=nn.Conv2d,kwargs={"in_channels":3, "out_channels":3, "kernel_size":1, "stride":1, "padding":0})
    _reluconvbn = Operation(operator=ReLUConvBN, kwargs={"out_channels":3, "kernel_size":3, "stride":1, "padding":1})

    _O = Categorical(choices=(_three, _one, _id))

    _C_ARGS = Categorical(
        choices=(
            (Resampled(_O),),
            (Resampled(_O), Resampled("model"), _reluconvbn),
            (Resampled(_O), Resampled("model")),
            (Resampled("model"),),
        ),
    )
    _C = Operation(
        operator=nn.Sequential,
        args=Resampled(_C_ARGS),
    )

    _model_ARGS = Categorical(
        choices=(
            (Resampled(_C),),
            (_reluconvbn,),
            (Resampled("model"),),
            (Resampled("model"), Resampled(_C)),
            (Resampled(_O), Resampled(_O), Resampled(_O)),
            (
                Resampled("model"),
                Resampled("model"),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
            ),
        ),
    )
    model = Operation(
        operator=nn.Sequential,
        args=Resampled(_model_ARGS),
    )
    a = 5

In [3]:
# Sampling and printing one random configuration of the pipeline

from neps.space.neps_spaces import neps_space

pipeline = NN_Space()
resolved_pipeline, resolution_context = neps_space.resolve(pipeline)

s = resolved_pipeline.model
s_config_string = neps_space.convert_operation_to_string(s)
pretty_config = neps_space.config_string.ConfigString(s_config_string).pretty_format()
s_callable = neps_space.convert_operation_to_callable(s)

print("Callable:\n")
print(s_callable)

print("\n\nConfig string:\n")
print(pretty_config)

Callable:

Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Identity()
)


Config string:

(<class 'torch.nn.modules.container.Sequential'> (<class 'torch.nn.modules.conv.Conv2d'> {'in_channels': 3, 'out_channels': 3, 'kernel_size': 3, 'stride': 1, 'padding': 1}) (<class 'torch.nn.modules.conv.Conv2d'>) (<class '__main__.Identity'>))
	01 :: <class 'torch.nn.modules.container.Sequential'>
		02 :: <class 'torch.nn.modules.conv.Conv2d'> {'in_channels': 3, 'out_channels': 3, 'kernel_size': 3, 'stride': 1, 'padding': 1}
		02 :: <class 'torch.nn.modules.conv.Conv2d'>
		02 :: <class '__main__.Identity'>


In [4]:
# Defining the pipeline, using the model from the NN_space space as callable

import numpy as np

def evaluate_pipeline(model: nn.Sequential):
    x = torch.ones(size=[1, 3, 220, 220])
    result = np.sum(model(x).detach().numpy().flatten())
    return result

In [73]:
import yaml
import neps
from neps.optimizers import load_optimizer
from pathlib import Path

import neps.space
import neps.space.neps_spaces
import neps.space.neps_spaces.neps_space
nn_space = NN_Space()
neps_state = neps.state.NePSState.create_or_load(path=Path("./results/compat/"),load_only=True)

print(load_optimizer(neps.algorithms.neps_random_search,nn_space)[0])


trial1 = neps_state.lock_and_sample_trial(optimizer=load_optimizer(neps.algorithms.neps_random_search,nn_space)[0], worker_id="1")
import pprint
config = neps.space.neps_spaces.neps_space.NepsCompatConverter().from_neps_config(trial1.config)
resolved_pipeline, resolution_context = neps_space.resolve(pipeline=NN_Space(),
    # Predefined samplings are the decisions made at each sampling step
    domain_sampler=neps_space.OnlyPredefinedValuesSampler(predefined_samplings=config.predefined_samplings),
    # Environment values are the fidelities and any arguments of the evaluation function not part of the search space
    environment_values=config.environment_values)
print(resolved_pipeline.get_attrs().keys())
print(resolution_context)

NePSRandomSearch()
dict_keys(['model', 'a'])


In [9]:
import neps

pipeline_space = NN_Space()

neps.run(
    evaluate_pipeline=evaluate_pipeline,
    pipeline_space=pipeline_space,
    optimizer=neps.algorithms.neps_random_search,
    root_directory="results/neps_spaces_nn_example",
    post_run_summary=True,
    max_evaluations_total=5,
    overwrite_working_directory=True,
)
neps.status("results/neps_spaces_nn_example", print_summary=True, pipeline_space_variables=(pipeline_space, ["model"]))

print("Done.")

# Configs: 5

    success: 5

# Best Found (config 3):

    objective_to_minimize: -12986.689453125
    config:
        (<class 'torch.nn.modules.container.Sequential'> (<class '__main__.Identity'>) (<class 'torch.nn.modules.conv.Conv2d'>) (<class 'torch.nn.modules.conv.Conv2d'>))
        	01 :: <class 'torch.nn.modules.container.Sequential'>
        		02 :: <class '__main__.Identity'>
        		02 :: <class 'torch.nn.modules.conv.Conv2d'>
        		02 :: <class 'torch.nn.modules.conv.Conv2d'>
    path: C:\Users\Amega\Git\neps\neps_examples\neps_spaces\results\neps_spaces_nn_example\configs\config_3
Done.
